In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from scipy.stats import ttest_ind
import statistics


In [ ]:
#importing the PRSice-2 output .best file (converetd into .txt in terminal). Here I add the sep to seperate the column names with a comma) 
#replace the df with vcdr gwas .best file: /mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/VCDR.best
df_prs = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/IOP.best', sep=r'\s+',header=0, encoding='ascii',engine='python')


In [ ]:
#importing the case control file 
df_cc = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/new_case_control.txt', sep=r'\s+',header=0, encoding='ascii',engine='python')

In [ ]:
count_ones = (df_cc['Phenotypes'] == 1).sum()

print("Number of occurrences of 1 in the 'Phenotype' column:", count_ones)

In [ ]:
4# Thought I should only use data thats used in the regression model
df_prs_filtered = df_prs[df_prs['In_Regression'] == 'Yes']
print(df_prs_filtered.head())

In [ ]:
df_prs_filtered = df_prs_filtered.dropna(subset=['PRS'])

In [ ]:
# convert PRS column to numeric
df_prs_filtered['PRS'] = pd.to_numeric(df_prs_filtered['PRS'], errors='coerce')

In [ ]:
#Merge the dfs
merged_df = pd.merge(df_prs_filtered, df_cc, on=['FID', 'IID'])

In [ ]:
# separate the data into cases and controls
cases = merged_df[merged_df['Phenotypes'] == 1]['PRS']
controls = merged_df[merged_df['Phenotypes'] == 0]['PRS']

In [ ]:
# plotting the normal distribution curves
plt.figure(figsize=(10, 6))
sns.histplot(cases, kde=True, label='Cases', color='orange', stat='density', common_norm=False)
sns.histplot(controls, kde=True, label='Controls', color='dodgerblue', stat='density', common_norm=False)

plt.legend(title='Group', labels=[ 'Cases','Controls'])


In [ ]:
# standardize the PRS data
scaler = StandardScaler()
merged_df['PRS_standardized'] = scaler.fit_transform(merged_df['PRS'].values.reshape(-1, 1))

# plot the standardized normal distribution curves with different colors and make it prety
plt.figure(figsize=(10, 6))
sns.histplot(data=merged_df, x='PRS_standardized', bins=30, hue='Phenotypes', kde=True, stat='density', common_norm=False)

plt.title('Standardized Normal Distribution of PRS for Cases and Controls for UKBiobank Data')
plt.xlabel('Standardized PRS')
plt.ylabel('Density')
plt.legend(title='Group', labels=[ 'Cases','Controls'])
plt.show()

In [ ]:
## NEXT - trying to make the propotion of individuals vs PRS decile graph

In [ ]:
# calculating the PRS deciles
merged_df['PRS_decile']= pd.qcut(merged_df['PRS_standardized'], q=10, labels=False)


In [ ]:
#just want to see how my df looks like
print (merged_df.head())

In [ ]:
# counting hte number of indv (cases vs controls)
decile_counts = merged_df.groupby(['PRS_decile', 'Phenotypes']).size().unstack(fill_value=0)


In [ ]:
# calculatin the propitons 
decile_proportions = decile_counts.div(decile_counts.sum(axis=1), axis=0)


In [ ]:
plt.figure(figsize=(12, 6))
barplot = sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases')
sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1])


for p, case_count in zip(barplot.patches, decile_counts[1]):
    height = p.get_height()
    ymin, ymax = plt.ylim()
    position = ymax - 0.99 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{case_count}',
                 ha='center')

# Annotate each bar with separate counts for controls
for p, control_count in zip(barplot.patches, decile_counts[0]):
    height = p.get_height() + decile_proportions[1]
    ymin, ymax = plt.ylim()
    position = ymax - 0.1 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{control_count}',
                 ha='center')
    
plt.title('Proportion of Cases and Controls in PRS Deciles with Counts')
plt.xlabel('PRS Decile')
plt.ylabel('Proportion')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

plt.figure(figsize=(18, 10))


In [ ]:
## OKay so ROC and AUC - 

X = merged_df[['PRS']]
y = merged_df['Phenotypes']

# Standardize the PRS data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

In [ ]:

#ROC curve
fpr, tpr, thresholds = roc_curve(y, X_standardized)

#AUC score
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()



In [ ]:
# T-test to see the difference b/w the means
#using ttest_ind because its not paired- cases and controls are independant 

t_statistic, p_value = ttest_ind(cases, controls, equal_var=False)
print(f'T-stat: {t_statistic:.4f}')
print(f'P-value: {p_value:.10g}')
p_value

#confused with the p-value? Thats too perfect 

In [ ]:
print (statistics.mean(cases))
print (statistics.mean(controls))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler

# Read PRSice-2 output files
df_prs_IOP = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/IOP.best', sep=r'\s+', header=0, encoding='ascii', engine='python')
df_prs_VCDR = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/VCDR.best', sep=r'\s+', header=0, encoding='ascii', engine='python')

# Read case control file
df_cc = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/new_case_control.txt', sep=r'\s+', header=0, encoding='ascii', engine='python')

# Filter and merge dataframes
def process_prs(df_prs, df_cc):
    df_prs_filtered = df_prs[df_prs['In_Regression'] == 'Yes'].dropna(subset=['PRS'])
    df_prs_filtered['PRS'] = pd.to_numeric(df_prs_filtered['PRS'], errors='coerce')
    merged_df = pd.merge(df_prs_filtered, df_cc, on=['FID', 'IID'])
    merged_df['PRS_decile'] = pd.qcut(merged_df['PRS'], q=10, labels=False)
    return merged_df

merged_IOP = process_prs(df_prs_IOP, df_cc)
merged_VCDR = process_prs(df_prs_VCDR, df_cc)

# Function to plot decile proportions
def plot_decile_proportions(ax, merged_df, title):
    decile_counts = merged_df.groupby(['PRS_decile', 'Phenotypes']).size().unstack(fill_value=0)
    decile_proportions = decile_counts.div(decile_counts.sum(axis=1), axis=0)
    sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases', ax=ax)
    sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1], ax=ax)
    for i, p in enumerate(ax.patches[:10]):
        ax.text(p.get_x() + p.get_width() / 2., p.get_height() - 0.02, f'{decile_counts[1][i]}', ha="center", fontsize=9)
    for i, p in enumerate(ax.patches[10:]):
        height = p.get_height() + decile_proportions[1][i]
        ax.text(p.get_x() + p.get_width() / 2., height - 0.05, f'{decile_counts[0][i]}', ha="center", fontsize=9)
    ax.set_xlabel('PRS Decile', fontsize=14)
    ax.set_ylabel('Proportion', fontsize=14)
 
    # Custom legend for Cases and Controls
    ax.legend([], frameon=False)
    ax.text(1.02, 0.1, 'Cases', color='darkorange', fontsize=14, va='center', ha='left', transform=ax.transAxes, rotation=90, fontweight='bold')
    ax.text(1.02, 0.8, 'Controls', color='skyblue', fontsize=14, va='center', ha='left', transform=ax.transAxes, rotation=90, fontweight='bold')


# Function to plot ROC curve
def plot_roc_curve(ax, merged_df, label):
    X = merged_df[['PRS']]
    y = merged_df['Phenotypes']
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(X)
    fpr, tpr, _ = roc_curve(y, X_standardized)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f'{label} AUC = {roc_auc:.2f}')
    return roc_auc

# Create the 1x3 facet plot
fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# Decile plots
plot_decile_proportions(axes[0], merged_IOP, 'IOP PRS Decile Plot')
plot_decile_proportions(axes[1], merged_VCDR, 'VCDR PRS Decile Plot')
axes[1].set_ylabel('') 

# ROC curve
axes[2].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
roc_auc_IOP = plot_roc_curve(axes[2], merged_IOP, 'IOP')
roc_auc_VCDR = plot_roc_curve(axes[2], merged_VCDR, 'VCDR')
axes[2].set_xlim([0.0, 1.0])
axes[2].set_ylim([0.0, 1.05])
axes[2].set_xlabel('False Positive Rate (FPR)', fontsize=14)
axes[2].set_ylabel('True Positive Rate (TPR)', fontsize=14)
axes[2].legend(title='GWAS',loc='lower right')

# Add panel labels
axes[0].text(-0.05, 1.05, '(A) IOP', transform=axes[0].transAxes, fontsize=16, fontweight='bold', va='top')
axes[1].text(-0.05, 1.05, '(B) VCDR', transform=axes[1].transAxes, fontsize=16, fontweight='bold', va='top')
axes[2].text(-0.05, 1.05, '(C)', transform=axes[2].transAxes, fontsize=16, fontweight='bold', va='top')

plt.tight_layout()

import os

folder_path = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Graph_Images/facet_plots'
file_path = os.path.join(folder_path, 'VCDR_IOP_GWAS.pdf')

# Ensure the folder exists and save the figure
os.makedirs(folder_path, exist_ok=True)
plt.savefig(file_path)

plt.close